ライブラリの読み込み

In [11]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

データの読み込み

In [5]:
INPUT_DIR = "C:/Users/sato-makoto/OneDrive - Hiroshima University/code/GCI/data/"

df = pd.read_csv(INPUT_DIR + "preprocessed.csv")
submittion = pd.read_csv(INPUT_DIR + "sample_submission.csv")
df.shape, submittion.shape

((232702, 138), (61500, 2))

dfをtrainとtestに分割

In [6]:
train = df[df["TARGET"].notnull()]
test = df[df["TARGET"].isnull()]
train.shape, test.shape

((171202, 138), (61500, 138))

trainを目的関数と説明変数に分割

In [7]:
# 目的変数と説明変数に分割
X = train.drop("TARGET", axis=1).values
y = train["TARGET"].values
test = test.drop("TARGET", axis=1).values

X.shape, y.shape, test.shape

((171202, 137), (171202,), (61500, 137))

学習

In [13]:
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

kf = KFold(n_splits = 10, shuffle = True)

roc_mean = []
test_pred = []

for train_index, valid_index in tqdm(kf.split(X), total=10):
  X_train = X[train_index]
  X_valid = X[valid_index]
  y_train = y[train_index]
  y_valid = y[valid_index]

  lgb = LGBMClassifier(random_state=0)
  lgb.fit(X_train, y_train)

  lgb_valid_pred = lgb.predict_proba(X_valid)[:, 1]
  roc_mean.append(roc_auc_score(y_valid, lgb_valid_pred))

  lgb_test_pred = lgb.predict_proba(test)[:, 1]
  test_pred.append(lgb_test_pred)

  0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
# XGBClassifier
from xgboost import XGBClassifier

kf = KFold(n_splits = 10, shuffle = True)

for train_index, valid_index in tqdm(kf.split(X), total=10):
  X_train = X[train_index]
  X_valid = X[valid_index]
  y_train = y[train_index]
  y_valid = y[valid_index]

  xgb = XGBClassifier(random_state=0)
  xgb.fit(X_train, y_train)
  
  xgb_train_pred = xgb.predict_proba(X_train)[:, 1]
  xgb_valid_pred = xgb.predict_proba(X_valid)[:, 1]
  print(f"Train Score: {roc_auc_score(y_train, xgb_train_pred)}")
  print(f"Valid Score: {roc_auc_score(y_valid, xgb_valid_pred)}")

  0%|          | 0/10 [00:00<?, ?it/s]

Train Score: 0.8707210289422385
Valid Score: 0.7374364371216026
Train Score: 0.8678236784729499
Valid Score: 0.7533451763506813
Train Score: 0.8649615640203004
Valid Score: 0.7525645864834621
Train Score: 0.8691373736019541
Valid Score: 0.7293139934694892
Train Score: 0.8691744349995207
Valid Score: 0.7460900817809408
Train Score: 0.8642768251094535
Valid Score: 0.7319237645083089
Train Score: 0.8626714148444891
Valid Score: 0.7435601826608507
Train Score: 0.864865312683323
Valid Score: 0.7342385746727849
Train Score: 0.8659180190703948
Valid Score: 0.7445685541699543
Train Score: 0.8674121992986333
Valid Score: 0.7442023818880522


ROCの平均

In [14]:
np.mean(roc_mean)

0.7493384914846328

予測結果を作成

In [15]:
submittion['TARGET'] = np.mean(test_pred, axis=0)
submittion

,SK_ID_CURR,TARGET
0,171202,0.027459
1,171203,0.146123
2,171204,0.126208
3,171205,0.099122
4,171206,0.196501
...,...,...
61495,232697,0.141894
61496,232698,0.045194
61497,232699,0.039691
61498,232700,0.105825


CSV出力

In [16]:
submittion.to_csv('pred.csv',index=False)